#Setting up

In [ ]:
!pip install diffusers transformers accelerate

In [ ]:
!accelerate config default

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import UniPCMultistepScheduler
from diffusers import StableDiffusionControlNetPipeline
from diffusers.utils import load_image
import PIL
import torch
import io
from google.colab import files

import cv2
import numpy as np
from PIL import Image

# Loading Model

In [ ]:
#@title **Hugging Face model**

model_repository = 'GreeneryScenery/SheepsControlV4' #@param {type:"string"}
stable_diffusion_repository = 'stabilityai/stable-diffusion-2-1-base' #@param {type:"string"}

controlnet = ControlNetModel.from_pretrained(model_repository, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    stable_diffusion_repository, controlnet=controlnet, torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# this command loads the individual model components on GPU on-demand.
pipe.enable_model_cpu_offload()

# Preprocess Image

In [ ]:
def canny(image):
  return Image.fromarray(cv2.Canny(np.asarray(image), 100, 200))

In [ ]:
# from PIL import Image
# import cv2
# import numpy as np
# canny(Image.open('10277.png'))

In [ ]:
!wget https://raw.github.com/s9xie/hed/master/examples/hed/deploy.prototxt
!wget https://vcl.ucsd.edu/hed/hed_pretrained_bsds.caffemodel

In [ ]:
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "hed_pretrained_bsds.caffemodel")
pad = 14

In [ ]:
def caffe_hed(image):
  img = image
  img = np.array(img)
  img = img[:, :, ::-1]
  img = cv2.copyMakeBorder(img, 50, 50, 50, 50, cv2.BORDER_CONSTANT, value = (255, 255, 255))
  (H, W) = img.shape[:2]
  blob = cv2.dnn.blobFromImage(img, scalefactor=1.0, size=(W, H),
      swapRB=False, crop=False)
  net.setInput(blob)
  hed = net.forward()
  hed = cv2.resize(hed[0, 0], (W, H))
  hed = (255 * hed).astype("uint8")
  blob = cv2.resize(blob[0, 0], (W, H))
  cropped_img = hed[H-256 - pad:H - pad, W-256 - pad:W - pad]
  return Image.fromarray(cropped_img)

#Generate image

In [ ]:
#@title **Calling model**

seed = 1 #@param {type:"slider", min:0, max:10000, step:1}
prompt = "Cute turtle" #@param {type:"string"}
# , trending on artstation, artstationHD, artstationHQ, patreon, 4k, 8k

generator = torch.manual_seed(seed)

filename = 'TurtlehappyhighlydetailedartgermartstationconceptartmattesharpfocusartbyWLOPandJamesJeanandVictoNgai_input.png' #@param {type:"string"}

preprocess = True #@param {type:"boolean"}
mode = "canny" #@param ["canny", "caffe_hed"]
upload = True #@param {type:"boolean"}
loaded = False #@param {type:"boolean"}

if loaded:
  init_image = PIL.Image.open(filename).convert("RGB")
elif not upload:
  #@markdown  Link and filename are ignored if uploading file
  repository = 'GreeneryScenery/SheepsControlV3' #@param {type:"string"}
  link = f'https://huggingface.co/{repository}/resolve/main/{filename}' #@param {type:"string"}
  !wget $link
  init_image = PIL.Image.open(filename).convert("RGB")
else:
  uploaded = files.upload()
  init_image = PIL.Image.open(io.BytesIO(uploaded[next(iter(uploaded))])).convert("RGB")

init_image.thumbnail((256, 256))

if preprocess:
  if mode == 'canny':
    init_image = canny(init_image)
  elif mode == 'caffe_hed':
    init_image = caffe_hed(init_image)

out_image = pipe(
    prompt, num_inference_steps=20, generator=generator, image=init_image
).images[0]

init_image.show()
out_image.show()

out_image.save(f"{prompt}.png")